# Feature Engineering and Classification for Jargon Words

In [15]:
from collections import Counter
from nltk.corpus import wordnet as wn
from nltk import word_tokenize

In [16]:
def generate_features(candidate):
    features = Counter()
    candidate = candidate.lower()
    features  = get_letter_combinations(candidate, features, 2)
    features  = get_letter_combinations(candidate, features, 3)
    features  = get_letter_combinations(candidate, features, 4)
    features['first'] = candidate[:1]
#     features['second'] = word[1:2] # get the 'h' in Charlie?
    features['last_three'] = candidate[-3:]
    features['last_two'] = candidate[-2:]
    features['first_three'] = candidate[:3]
    features['name_len_id'] = len(candidate)
#     features['repeating_letters'] = get_first_repeating_letters(word)
#     features['continuous_vowels'] = get_first_repeating_vowels(word)
#     features['has_letters'] = has_letters(word, 'yzwx')
    return dict(features)
generate_features('veni vidi vici') 

['veni', 'vidi', 'vici']

In [8]:
def get_letter_combinations(candidate, features, number):
    if len(candidate) < number:
        return features
    else:
        for index in range(0, len(candidate), number):
            features[candidate[index:index + number]] += 1
        return features

In [ ]:

def get_wordnet_definition(candidate):
    words = word_tokenize(candidate)
    for word in words:
        synsets = wn.synsets(word)
        
    
    